<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/TwitterAPIv2_get_following_and_save_google_spreadsheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Twitterで自分がフォローしているフォロー先一覧を取得してCSV及び、Googleスプレッドシートに保存するPythonスクリプト

前提条件
TwitterのAPIを使う為に開発者登録をする方法(2021年版)
(https://webbigdata.jp/page-640/page-11005/page-10971)
を参考にTwitterのDeveloper Account登録をしてBearer Tokenを取得済である事

大まかな流れ

(1)自分のusername(@hogeのhogeの部分)からuser_id(ユーザを一意に特定する番号)を得る

(2)user_idとBearer Tokenを使って自分がフォローしているユーザの一覧を得てCSVに保存する

(3)CSVから取得したフォロー先ユーザの情報をGoogle Spread Sheetに保存する


## (1)自分のusernameからuser_idを取得

参考情報

Twitter社が提供している公式サンプル

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Lookup/get_users_with_bearer_token.py

In [ ]:
import requests
import os
import json

# ★TODO：自分のBearer Tokenに差し替えてください。
bearer_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx""

def create_url():
    # ★TODO：XXXXXXを自分のusernames(@hogehogeのhogehogeの部分)に差し替えてください。
    usernames = "usernames=XXXXXXX"
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))

if __name__ == "__main__":
    main()

'''
実行すると、以下のように表示されるので"id": "xxxxxxxxxxxxxxxxxxxx"のxxxxxxxxxxxxの部分をメモっておく
200
{
    "data": [
        {
            "created_at": "2021-06-08T12:53:07.000Z",
            "description": "xxxxxxxxxxxxxxxxxxx",
            "id": "xxxxxxxxxxxxxxxxxxxxxxxxx",
            "name": "XXXXX",
            "username": "XXXX"
        }
    ]
}
'''

## (2)user_idとBearer Tokenを使ってフォロー先のユーザ情報を得る

参考情報
Twitter社が提供している公式サンプル

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Follows-Lookup/followers_lookup.py

https://developer.twitter.com/en/docs/twitter-api/users/follows/api-reference/get-users-id-following


In [ ]:
import requests
import os
import json
import pandas as pd

# ★TODO：自分のBearer Tokenに差し替えてください。
bearer_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

def create_url():
    # ★TODO：xxxxxxxxxxxxxの部分を(1)で取得した自分のuser_idに差し替えてください。
    # また、このAPIは連続して実行すると
    # 「Request returned an error: 429 Rate limit exceeded」とエラーになる事があります。
    user_id = xxxxxxxxxxxxxxxxxxx
    return "https://api.twitter.com/2/users/{}/following".format(user_id)

def get_params(pagination_token=""):
    if pagination_token != "":
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000", "pagination_token": pagination_token}
    else:
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000"}

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def save_range_list(json_response, range_list):
  cell_list = []
  for v in json_response['data']:
    # userを一意に特定できる不変のID
    cell_list.append(v['id'])
    # プロフィール画面などに出て来る名称
    cell_list.append(v['name'])
    # Twitterアカウントの@hogehogeのhogehogeの部分
    cell_list.append(v['username'])
    cell_list.append(v['public_metrics']['followers_count'])
    cell_list.append(v['public_metrics']['listed_count'])
    cell_list.append(v['public_metrics']['tweet_count'])
    range_list.append(cell_list)
    cell_list = []

def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    #print(json.dumps(json_response, indent=4, sort_keys=True))

    range_list = [['id', 'name', 'username', 'followers_count', 'listed_count','tweet_count']]
    save_range_list(json_response, range_list)
    
    while (json_response.get("meta") and json_response.get("meta").get("next_token")):
      params = get_params(json_response['meta']['next_token'])
      json_response = connect_to_endpoint(url, params)
      save_range_list(json_response, range_list)

    # /content/配下に保存する
    df = pd.DataFrame(range_list)
    df.to_csv("/content/df_following.csv", header=False, index=False)


if __name__ == "__main__":
    main()

## (3)Google スプレッドシートに保存する

参考情報：
Google スプレッドシートを操作するためのライブラリgspread
https://docs.gspread.org/en/latest/index.html

In [ ]:
import gspread
import pandas as pd

In [ ]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# 「スプレッドシートの有無を確認して存在したら開き、存在しなかったら作る」事をするためには
# Google ドライブへのアクセス権限(Drive API)が必要なのですが、権限回りは古い情報が混在していて
# どれが正しいのか裏が取れていないため、雑ですがファイルを開こうとして例外が発生したらスプレッドシートを作成しています。
workbook = ""
try:
  workbook = gc.open('Twitter_user_list_following')
except BaseException as e:
  workbook = gc.create('Twitter_user_list_following')

# CSVの読み込み
df = pd.read_csv("/content/df_following.csv")
worksheet = workbook.get_worksheet(0)

# idを数値のままにしておくと巨大すぎてGoogleスプレッドシートで末尾が000に切り捨てられてしまうため
df = df.astype({'id': str})
worksheet.update([df.columns.values.tolist()] + df.values.tolist())



スプレッドシートは(https://drive.google.com/drive/my-drive)に出来るのですが、
Colabのランタイムをリセットしないとゴミ箱に入れたファイルに書き込んだりして、
どのスプレッドシートに書き込まれたのかわからなくなってしまったり、
Requested entity was not foundとエラーが出る事があります。
その場合はランタイムを再起動したり、以下の方法で捜し出してください。

上のセルを実行した際に
{'spreadsheetId': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
・・・と表示されると思いますが、
スプレッドシートのURLのyyyyyyyyyyyyyyyyyyyyyyyyyyyの部分を置き換えると直接アクセスできます。
https://docs.google.com/spreadsheets/d/yyyyyyyyyyyyyyyyyyyyyyyyyyy/edit#gid=0
